# ESTIMATION OF WINE QUALITY USING THE CHEMICAL ANALYSIS DATA

**Introduction**

The wine quality determines the price of the wine. The quality is measured from every aspects such as the vintage, variety, type, place , the grape used, and or the various chemical analysis. Each type of wine has its own composition and taste. 

There are about five types of wine: Red wine, White wine, Rose wine, Sparkling wine and Fortified wine.Since there are various varieties and blends under each type, the quality estimation has become difficult. In this report, we want to derive a new method of estimating quality using chemical analysis.


![alt text](https://i.pinimg.com/originals/f8/69/cc/f869cc87b2246a31b480630de573aede.png)


**Datasets**

We have taken two datasets:

1)  Wine Quality by: Paulo Cortez (Univ. Minho), Antonio Cerdeira, Fernando Almeida, Telmo Matos and Jose Reis (CVRVV) @ 2009


[Wine chemical datasets]

In [ ]:
import pandas as pd
import numpy as np

chemical1 = pd.read_csv('../input/wine-quality/winequalityN.csv')
print(chemical1.shape)
chemical1.info()

2) Wine Review: The data was scraped from WineEnthusiast during the week of June 15th, 2017.

[Wine review datasets]

In [ ]:
sensory1 = pd.read_csv('../input/wine-reviews/winemag-data_first150k.csv',engine='python', error_bad_lines=False)
print(sensory1.shape)
sensory1.info()

In [ ]:
sensory2 = pd.read_csv('../input/wine-reviews/winemag-data-130k-v2.csv',engine='python', error_bad_lines=False)
print(sensory2.shape)
sensory2.info()

# Analysis of wine chemical dataset

 First, we will divide the dataset for both white wine and red wine.

In [ ]:
import matplotlib.pyplot as pit


total = len(chemical1.index)
white_wine = chemical1[chemical1['type']=='white']
print("Percentage of white wine: ",(len(white_wine.index)/total)*100,"%")
red_wine = chemical1[chemical1['type']=='red']
print("Percentage of red wine: ",(len(red_wine.index)/total)*100,"%")

pit.pie([len(white_wine.index),len(red_wine.index)], colors = ['#635e6b','#EF5350'], labels = ['white wine','red wine'],startangle=90)
pit.title('Types in the wine chemical datasets')
pit.show()

The number of wines in each quality point of both red and white wine are:

In [ ]:
from prettytable import PrettyTable

labels = ['Quality 1','Quality 2','Quality 3','Quality 4','Quality 5','Quality 6','Quality 7','Quality 8','Quality 9','Quality 10']
t = PrettyTable(['Quality', 'white wine', 'red wine'])
values_white = []
values_red = []
for i in range(1,11):
  values_white.append(len(white_wine[white_wine['quality']==i]))
  values_red.append(len(red_wine[red_wine['quality']==i]))
  t.add_row([i,len(white_wine[white_wine['quality']==i]),len(red_wine[red_wine['quality']==i])])

print('Wines in each quality')
print(t)

colors_w = ['#fdf7ff','#f8f2ff','#f1ebfa','#e2ddec','#bfbac8','#a09ba9','#77727f','#635e6b','#433f4b','#221e29']
colors_r = ['#FFEBEE','#FFCDD2','#EF9A9A','#E57373','#EF5350','#F44336','#E53935','#D32F2F','#C62828','#B71C1C']

fig,(ax1,ax2)= pit.subplots(2,1)
fig.set_size_inches(10,9)
#ax1.pie(values_white, colors = colors_w, labels=labels, startangle=90)
ax1.bar(labels, values_white)
ax1.set_title('Number of white wines in each quality')
ax1.legend()

#ax2.pie(values_red, colors = colors_r, labels=labels, startangle=90)
ax2.bar(labels, values_red)
ax2.set_title('Number of white wines in each quality')
ax2.set_title('Number of red wines in each quality')
ax2.legend()

pit.show()


Let's analysis this data by finding the minimum and maximum value of each chemical attribute for every quality points both in the case of red and white wine.

In [ ]:
import seaborn as sns

feature_names = ['fixed acidity','volatile acidity', 'citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']

def find_MIN_MAX(df,q):
  print("quality: ", q)
  if(df.empty):
    print('no datasets')
    return
  else:
    t = PrettyTable(['Name', 'min', 'max'])
    min_list=[]
    max_list=[]
    for x in range(1,12):
      name = df.columns[x]
      mini = df[df.columns[x]].min()
      min_list.append(mini)
      maxi = df[df.columns[x]].max()
      max_list.append(maxi)
      t.add_row([name,mini,maxi])
    print(t)
    barWidth = 0.3
    pit.figure(figsize=(18, 5))
    r1 = np.arange(len(min_list))
    r2 = [x + barWidth for x in r1]
    pit.bar(r1, min_list, width = barWidth, color = 'blue', edgecolor = 'black',label='Min')
    pit.bar(r2, max_list, width = barWidth, color = 'cyan', edgecolor = 'black', label='Max')
    pit.xticks([r + barWidth for r in range(len(min_list))], feature_names)
    pit.ylabel('Values')
    pit.legend()
    pit.show()

    

print('Red Wine Data:')

for i in range(1,11):
   find_MIN_MAX(red_wine[red_wine['quality']==i],i)



print('White Wine Data:')

for i in range(1,11):
  find_MIN_MAX(white_wine[white_wine['quality']==i],i)


Since the quality points mentioned here is in the scale of 0-10, we will change it into scale of 100.

NewValue = (((OldValue - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin

In [ ]:
#for white wine
old_min = white_wine['quality'].min()
old_max = white_wine['quality'].max()

new_min = sensory1['points'].min()
new_max = sensory1['points'].max()
print(new_max)

white_wine['points'] = (((white_wine['quality'] - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min
white_wine.head()

In [ ]:
#for red wine
old_min = red_wine['quality'].min()
old_max = red_wine['quality'].max()

red_wine['points'] = (((red_wine['quality'] - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min
red_wine.head()

# Analysis of wine reviews datasets

The wine reviews are seperated into two different csv files. 

In [ ]:
uniq_variety1 = sensory1['variety'].unique()
uniq_vriety2 = sensory2['variety'].unique()
common_variety = list(set(uniq_variety1).intersection(uniq_vriety2))
print('Common and unique varieties in both csv are: ',common_variety)
print('Number of varieties :',len(common_variety))

Since we have many varieties of wine in each of the five categories and we have datasets only for white and red wine, we will consider only the varieties of white and red wines.

In [ ]:
white_varieties = ['Aligoté','Alvarinho', 'Auxerrois', 'Bacchus','Bual','Chardonnay','Chasselas','Chenin','Blanc','Colombard','Emerald','Riesling','Fumé','Blanc','Folle','Blanche','Furmint','Gewürztraminer','Grüner Veltliner','Hárslevelü','Jacquère','Kerner','Malvasia','Marsanne','Morio-Muscat','Müller-Thurgau','Muscadelle','Muscadet','Moscato','Palomino','Pedro Ximenez','Picolit','Pinot Blanc','Pinot Gris','Riesling','Rkatsiteli','Sacy','Savagnin','Sauvignon Blanc','Scheurebe','Sémillon','Sercial','Seyval Blan','Silvaner','Trebbiano','Verdelho','Verdicchio','Vidal','Viognier','Viura','Welschriesling']
red_varieties = ['Aglianico','Alicante','Baco','Noir','Barbera','Cabernet Franc','Cabernet Sauvignon','Carignan','Cinsault','de Chaunac','Dolcetto','Freisa','Gamay','Gamay Beaujolais','Grenache','Grignolino','Kadarka','Lambrusco','Malbec','Maréchal Foch','Merlot','Mourvèdre','Nebbiolo','Petite Sirah','Pinot Noir','Pinot','Meunier','Pinotae','primitivo','Ruby Cabernet','Sangiovese','Syrah','Tempranillo,''Touriga Naçional','Xynomavro','Zinfandel']
print('Number of white wine varieties considered: ', len(white_varieties))
print('Number of red wine varieties considered: ', len(red_varieties))

Abstracting only those varieties: ( since some rows have no price and also price is an important attribute, we ignore those rows)

In [ ]:
white = pd.concat([sensory1[sensory1['variety'].isin(white_varieties)] ,sensory2[sensory2['variety'].isin(white_varieties)]],sort=False)
white = white[np.isfinite(white['price'])]
print('shape of the white wine: ', white.shape)
white.head()

In [ ]:

red = pd.concat([sensory1[sensory1['variety'].isin(red_varieties)],sensory2[sensory2['variety'].isin(red_varieties)]],sort=False)
red = red[np.isfinite(red['price'])]
print('shape of the red wine: ', red.shape)
red.head()

# Predicting the price of the wine based on sensory points

From the wine reviews data, we can create a model for price against points(sensory points). We use Linear Regression to create the model.



In [ ]:
from sklearn.linear_model import LinearRegression,LogisticRegression,  BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn import metrics

x = white[['points']]
y = white[['price']]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)
model_price_white = LinearRegression()
model_price_white.fit(x_train,y_train)
y_pred = model_price_white.predict(x_test)
r =metrics.mean_squared_error(y_test,y_pred)
print("white wine::")
print("MEAN SQUARE ERROR: ", r)
print("ROOT MEAN SQUARE ERROR: ",np.sqrt(r))

pit.scatter(x,y)
pit.xlabel('quality points')
pit.ylabel('price')
pit.title('Price determined from Quality points for white wine')
pit.show()


In [ ]:
x = red[['points']]
y = red[['price']]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)
model_price_red = LinearRegression()
model_price_red.fit(x_train,y_train)
y_pred = model_price_red.predict(x_test)
r =metrics.mean_squared_error(y_test,y_pred)
print("red wine::")
print("MEAN SQUARE ERROR: ", r)
print("ROOT MEAN SQUARE ERROR: ",np.sqrt(r))

pit.scatter(x,y,color='r')
pit.xlabel('quality points')
pit.ylabel('price')
pit.title('Price determined from Quality points for red wine')
pit.show()

Now finding the unknown price of the wine in the wine quality dataset.

In [ ]:
white_wine['price']=model_price_white.predict(np.array(white_wine['points']).reshape(-1,1))
red_wine['price']=model_price_red.predict(np.array(red_wine['points']).reshape(-1,1))

# Estimation of quality based on chemical properties

Now we focus on the wine chemical datasets. We train a classification algorithm (Random Forest) with all the chemical properties against the sensory quality. The reason we take the sensory quality as the target is that the wine's taste greatly depends on tose properties. The model for the white wine:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns

x_white = white_wine[feature_names]
y = white_wine['quality']
x_white.fillna(0, inplace=True)


We split the dataset into train and test sets.
    train sets: 70%
    test sets: 30%

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_white,y,test_size=0.3)


Training the model:

In [ ]:
model = RandomForestClassifier(n_estimators=10)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

Finding the precission recall metrics and ploting the graph which gives the importance of each property.

In [ ]:
print("PRECISION RECALL: ", metrics.recall_score(y_test,y_pred,average='weighted'))

feature_imp = pd.Series(model.feature_importances_,index = feature_names).sort_values(ascending=False)
print(feature_imp)

sns.barplot(x=feature_imp,y=feature_imp.index)
pit.xlabel('Properties importance score')
pit.ylabel('Chemical Properties')
pit.title('Visualizing Important Features for white wine')
pit.legend()
pit.show()

This is for red wine:

In [ ]:
x_red = red_wine[feature_names]
y = red_wine['quality']
x_red.fillna(0, inplace=True)

x_train,x_test,y_train,y_test = train_test_split(x_red,y,test_size=0.3)

model = RandomForestClassifier(n_estimators=10)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

print("PRECISION RECALL: ", metrics.recall_score(y_test,y_pred,average='weighted'))

feature_imp = pd.Series(model.feature_importances_,index = feature_names).sort_values(ascending=False)
print(feature_imp)

sns.barplot(x=feature_imp,y=feature_imp.index)
pit.xlabel('Properties importance score')
pit.ylabel('Chemical Properties')
pit.title('Visualizing Important Features for red wine')
pit.legend()
pit.show()

Taking the first five important features from both red wine and white wine random forest models, to create a logistic regression model for predicting the quality.

In [ ]:
feature = ['alcohol','density','volatile acidity','free sulfur dioxide','total sulfur dioxide','sulphates']
x = white_wine[feature]
y = white_wine['quality']
x.fillna(0, inplace=True)

model_white = LogisticRegression() #RandomForestClassifier(n_estimators=10)
model_white.fit(x,y)

x = red_wine[feature]
y = red_wine['quality']
x.fillna(0, inplace=True)

model_red = LogisticRegression() #RandomForestClassifier(n_estimators=10)
model_red.fit(x,y)

Now lets find the quality based on this model for the wine quality datasets.

In [ ]:
white_wine['predicted quality'] = model_white.predict(x_white[feature])
red_wine['predicted quality'] = model_red.predict(x_red[feature])

#Determining the price for the estimated quality

We use the same model(Linear regression) that was trained with points and price for calculating the price again. Before we use the model, first lets convert the quality predicted from the scale of 0-10 to 0-100 scale. This is for white wine:

In [ ]:
white_wine['predicted points'] = (((white_wine['predicted quality'] - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min
white_wine['predicted price'] = model_price_white.predict(white_wine[['predicted points']])

This is for red wine:

In [ ]:
red_wine['predicted points'] = (((red_wine['predicted quality'] - old_min) * (new_max - new_min)) / (old_max - old_min)) + new_min
red_wine['predicted price'] = model_price_red.predict(red_wine[['predicted points']])

# Comparing the results

In [ ]:
total_price_white = sum(white_wine['price'])
total_price_red = sum(red_wine['price'])
total_price = total_price_white + total_price_red

print('Predicted price from sensory points')
print("Total price for white wine: ", "{0:.2f}".format(total_price_white))
print("Total price for red wine: ", "{0:.2f}".format(total_price_red))
print("Total price : ", "{0:.2f}".format(total_price))


predicted_total_price_white = sum(white_wine['predicted price'])
predicted_total_price_red = sum(red_wine['predicted price'])
predicted_total_price = predicted_total_price_white + predicted_total_price_red

print('Predicted price from chemical analysis')
print("Total predicted price for white wine: ", "{0:.2f}".format(predicted_total_price_white))
print("Total predicted price for red wine: ", "{0:.2f}".format(predicted_total_price_red))
print("Total predicted price : ", "{0:.2f}".format(predicted_total_price))

barWidth = 0.2
pit.figure(figsize=(8, 6))
r1 = np.arange(3)
r2 = [x + barWidth for x in r1]
pit.bar(r1, [total_price_white,total_price_red,total_price], width = barWidth, color = 'blue', edgecolor = 'black',label='Predicted price from sensory points')
pit.bar(r2, [predicted_total_price_white,predicted_total_price_red,predicted_total_price], width = barWidth, color = 'cyan', edgecolor = 'black', label='Predicted price from chemical analysis')
pit.xticks([r + barWidth for r in range(3)], ['white wine','red wine','total'])
pit.ylabel('Price')
pit.title('Comparing the price calculated from sensory points and estmated quality points from chemical analysis')
pit.legend()
pit.show()

From the above, we can say that the wine makers can have a profit for white wine and loss for red wine, but has a total profit.

In [ ]:
print('Profit gained for white wine: ',((predicted_total_price_white-total_price_white)/total_price)*100,"%")
print('Loss gained for red wine: ',((total_price_red-predicted_total_price_red)/total_price)*100,"%")
print('Total Profit gained: ',((predicted_total_price-total_price)/total_price)*100,"%")

# Conclusion

Sensory points are awarded by tasting the wine. This is not standard and will vary among many tasters. Hence estimating its quality from the sensory points is not accurate.

Our method estimates the quality from the chemical properties, which even the computer can calculate.The price can be changed accourding to the winery, if they can train the computer with their own quality vs price datasets.

# References



1.   [Red and White Wine Analysis](https://https://rstudio-pubs-static.s3.amazonaws.com/98369_7d87780667b74901af21ff93c1c1e1db.html), by Ian Xiao on June 27, 2015
2.  [Correlating Wine Quality Indicators to Chemical and Sensory](https://www.mdpi.com/1420-3049/20/5/8453/pdf
)by Helene Hopfer, Jenny Nelson, Susan E. Ebeler and Hildegarde Heymann on Molecules 2015, 20
3.[ How to Choose a Good Wine](https://www.marketviewliquor.com/blog/2018/08/how-to-choose-a-good-wine/)

**Datasets:**


1.   [Wine Quality dataset](https://www.kaggle.com/rajyellow46/wine-quality) downloaded from kaggle provided by Raj Kumar
2.  [Wine Review](https://www.kaggle.com/zynicide/wine-reviews) downloaded from kaggle provided by zackthout





